In [4]:
! pip install dspy sentence-transformers

  Using cached dspy-3.0.4-py3-none-any.whl.metadata (8.4 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached openai-2.13.0-py3-none-any.whl.metadata (29 kB)
  Using cached jiter-0.12.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 153.2 MB/s  0:00:00
Using cached openai-2.13.0-py3-none-any.whl (1.1 MB)
Using cached jiter-0.12.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (361 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 122.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [dspy]2m11/12 [dspy]nce-transformers]


In [18]:
import dspy
from sentence_transformers import SentenceTransformer
from dspy.teleprompt import KNNFewShot

In [19]:
lm = dspy.LM(
    model="openai/TheBloke/CodeLlama-7B-Instruct-AWQ",
    api_base="http://localhost:8000/v1",
    api_key="dummy",
    max_tokens=1024,
    temperature=0.0
)
dspy.configure(lm=lm) 

In [27]:
import dspy
import numpy as np
from dspy.teleprompt import KNNFewShot
from sentence_transformers import SentenceTransformer

qa = dspy.ChainOfThought("question -> answer")

trainset = [
    dspy.Example(question="What is the capital of France?",continent="Europe", answer="Paris").with_inputs("question","continent"),
    dspy.Example(question="What is the capital of Pakistan?",continent="Asia", answer="Islamabad").with_inputs("question","continent"),
    dspy.Example(question="What is color of sky?",continent="None", answer="blue").with_inputs("question","continent"),
    dspy.Example(question="What is capital of Germany?",continent="Europe", answer="Berlin").with_inputs("question","continent")
]

# Must return a numpy array, NOT a list
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def vectorizer_func(texts: list[str]):
    print(texts[0])
    return embedding_model.encode(texts) 

# 5. Initialize KNNFewShot
# We pass the trainset HERE. It is stored inside the object.
knn_teleprompter = KNNFewShot(
    k=2, 
    trainset=trainset, 
    vectorizer=vectorizer_func
)

compiled_qa = knn_teleprompter.compile(qa)

# 7. Run
result = compiled_qa(question="What is the capital of Belgium?",continet="Europe")
print(f"Predicted Answer: {result.answer}")

question: What is the capital of France? | continent: Europe
question: What is the capital of Belgium? | continet: Europe


100%|██████████| 2/2 [00:00<00:00, 11.96it/s]


Bootstrapped 2 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Predicted Answer: Brussels


In [21]:
lm.inspect_history(n=1)





[2025-12-18T21:36:21.174411]

System message:

Your input fields are:
1. `question` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
What is the capital of France?


Assistant message:

[[ ## reasoning ## ]]
]]
The capital of France is Paris.

[[ ## answer ## ]]
Paris

[[ ## completed ## ]]


User message:

[[ ## question ## ]]
What is capital of Germany?


Assistant message:

[[ ## reasoning ## ]]
]]
The capital of Germany is Berlin.

[[ ## answer ## ]]
Berlin

[[ ## completed ## ]]


User message:

[[ ## question ## ]]
What is the capital of Belgium?

Respond with the corresponding output fi

In [28]:
import re

class TextToSQL(dspy.Signature):
    """Convert natural language to SQL based on the given schema."""
    db_schema = dspy.InputField() 
    question = dspy.InputField()
    sql = dspy.OutputField()

# 3. Define the Training Data (The "Not Schema" Part)
# CRITICAL TRICK: We intentionally OMIT 'db_schema' from these examples.
# DSPy will simply print "Question: ... SQL: ..." in the prompt, which is what you want.
# We also set .with_inputs("question") so the Indexer knows to focus on that.
trainset = [
    dspy.Example(question="How many users are there?", sql="SELECT count(*) FROM users").with_inputs("question"),
    dspy.Example(question="Find the email of user 1", sql="SELECT email FROM users WHERE id = 1").with_inputs("question"),
    dspy.Example(question="List all product names", sql="SELECT name FROM products").with_inputs("question"),
]

# 4. Define Custom Vectorizer (The "Embedding" Part)
# This handles the Test Time issue where 'db_schema' is passed but we must ignore it.
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def questions_only_vectorizer(texts: list[str]):
    cleaned_texts = []
    for text in texts:
        # DSPy formats inputs like: "db_schema: ... | question: ..."
        # We use regex to find "question: " and take everything after it.
        match = re.search(r"question:\s*(.+)", text, re.IGNORECASE)
        if match:
            # Found the question part! Embed only this.
            cleaned_texts.append(match.group(1))
        else:
            # Fallback: if 'question:' isn't found (rare), embed the whole thing
            cleaned_texts.append(text)
            
    return embedding_model.encode(cleaned_texts)

# 5. Initialize & Compile
# We pass our custom vectorizer here
knn_teleprompter = KNNFewShot(
    k=2, 
    trainset=trainset, 
    vectorizer=questions_only_vectorizer
)

module = dspy.ChainOfThought(TextToSQL)
compiled_sql = knn_teleprompter.compile(module)

# 6. Run it
# At test time, we MUST provide db_schema, but our vectorizer will ignore it
my_schema = "CREATE TABLE users (id INT, name TEXT, email TEXT);"
query = "What is the email for user id 5?"

result = compiled_sql(db_schema=my_schema, question=query)

# 7. Verification
print(f"Generated SQL: {result.sql}")
print("\n--- Inspecting the Prompt ---")
# You will see the schema is in the 'Main' part, but the Few-Shot examples ONLY have Question/SQL
lm.inspect_history(n=1)

100%|██████████| 2/2 [00:27<00:00, 13.57s/it]


Bootstrapped 2 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Generated SQL: SELECT email FROM users WHERE id = 5

--- Inspecting the Prompt ---




[2025-12-18T21:55:16.676736]

System message:

Your input fields are:
1. `db_schema` (str): 
2. `question` (str):
Your output fields are:
1. `reasoning` (str): 
2. `sql` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## db_schema ## ]]
{db_schema}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## sql ## ]]
{sql}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Convert natural language to SQL based on the given schema.


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## question ## ]]
Find the email of user 1


Assistant message:

[[ ## reasoning ## ]]
]]
To find the email of user 1, we need to query the `users` table and select the `email` 